# Cryptographie : TD 4

Dans ce TD nous étudierons des algorithmes reliés à la cryptographie basée sur les réseaux.

## 1) Algorithme LLL

Soit $\mathcal{L} \subseteq \mathbb{R}^n$ un réseau de rang complet. Notons $\lambda_1(\mathcal{L})$ la longeur du plus court vecteur dans $\mathcal{L}$. Le problème de trouver $\lambda_1(\mathcal{L})$ est appelé SVP (Shortest Vector Problem). Plus généralement, pour $r \geq 1$ on appelle $\delta-$SVP au problème de trouver un vecteur $v \in \mathcal{L}$ dont la norme satisfait $||v|| \leq r \lambda_1(\mathcal{L})$. L'algorithme de LLL (Lenstra, Lenstra et Lovász) permet, pour $1/4 < \delta < 1$, de trouver un vecteur $v$ dans un réseau $\mathcal{L}$ tel que $||v|| \leq \left(\frac{2}{\sqrt{4 \delta - 1}} \right)^{n-1} \lambda_1(\mathcal{L})$. En particulier, pour $\delta = 3/4$ on obtient une solution au $r$-SVP avec $r = 2^{\frac{n-1}{2}}$.

a) Rappeler la définition de $\delta-$LLL base réduite d'un réseau et l'algorithme LLL.

b) Implémenter l'algorithme LLL.

In [ ]:
from numpy import*
# LLL

# La fonction Gram_Schmidt prend comme argument un vecteur b et retourne le vecteur bb qui obtenu via l'orthonormalisation de Gram-Schmidt
def Gram_Schmidt(b) :
  #print(b, len(b))
  bb = [b[0]]
  for i in range(1,len(b)):
    u_ij = [0 for k in range(len(b[0]))]
    for j in range(i):
      u_ij = u_ij + ( dot(b[i],bb[j]) / dot(bb[j],bb[j]) )*bb[j]
      
    bb=insert([b[i]-u_ij],len(bb)-1,bb,axis=0)

  return bb

b=[9,22]

a=array([[2,5],b])

print(Gram_Schmidt(a))
c=array([[10,0,50],[0,1000,7],[-5.,50,17]])
print(Gram_Schmidt())


In [71]:
# La fonction LLL prend comme argument une base d'un réseau et un nombre 1/4 < delta < 1 et retourne une base delta-LLL reduite

def LLL(base, delta) :
  b=Gram_Schmidt(base)
  b_r = base.copy()
  for i in range(1,len(base)):
    for j in range(i-1,1,-1):
        c_ij= round(dot(base[i],b[j])/dot(b[j],b[j]))
        b_r[i]=base[i]-c_ij*base[j]
        #print(b_r)

  for i in range(len(base)-1):
    u= dot(b_r[i+1],b[i])/dot(b[i],b[i])
    if (delta*square(linalg.norm(b[i],ord=2))>linalg.norm(u*b[i+1]+b[i],ord=2)):
      t=b_r[i].copy()
      b_r[i]=b_r[i+1]
      b_r[i+1]=t
  return b_r

print(LLL(c,3/4))



[[ 0.00000000e+00  1.00000000e+03  7.00000000e+00]
 [-5.00000000e+00  5.00000000e+01  1.70000000e+01]
 [-5.00000000e+00 -4.83474666e+18 -3.38432266e+16]]


c) An appliquant l'algorithme précédent, calculer des bases reduites avec $\delta = 3/4$ pour les bases données par les vecteurs suivants :

* $b_1 = (9, 22), \;\;\; b_2 = (2, 5)$.
* $ b_1 = (10, 0, 50) , \; \; \; b_2 = (0, 1000, 7), \; \; \; b_3 = (-5, 50, 17)$

## 1.2) Deux applications de LLL

Une des applications importantes de l'algorithme LLL est au problème de trouver des relations linéaires etières entre des nombres réels. Soient $x_1, \ldots, x_n \in \mathbb{R}$. On veut savoir si ils existent $\lambda_1, \ldots, \lambda_n \in \mathbb{Z}$, pas tous nuls et tels que

$ \lambda_1 x_1 + \ldots + \lambda_n x_n = 0$.

Si des tels $(\lambda_i) \in \mathbb{Z}^n - \{ 0 \}$ existent, on dit que les éléments $(x_i)$ sont entièrement linéairement dépendents.
Pour cela, on peut considérer, pour un entier $k \in \mathbf{N}$, le réseau $\mathcal{L}$ de rang $n$ dans $\mathbb{R}^{n+1}$ de base

$b_1 = (1, 0, \ldots, 0, 10^k x_1), b_2 = (0, 1, \ldots, 0, 10^k x_2), \ldots, b_n = (0, \ldots, 0, 1, 10^k x_n).$

a) Montrer que les éléments $(x_i)$ sont entièrement linéairement dépendents si et seulement si le réseau $\mathcal{L}$ (pour n'importe quel $k$) contient un vecteur non nul dont la dernière coordonnée est nulle.

b) Soit $x \in \mathbb{R}$. Rappelons que $x$ est algébrique si il existe un polynome $P(T) \in \mathbf{Z}[T]$ tel que $P(x) = 0$. Trouver un algorithme pour vérifier si l'élément $x$ est algébrique en utilisant LLL.

c) Écrire des implémentations des algorithmes pour résoudre les problèmes de dépendence entière linéaire et d'algébricité des nombres réels.

# Réponses

a)En considérant un vecteur non nul $\lambda_1b_1+\lambda_2b_2+...\lambda_nb_n$ du réseau $\mathcal{L}$. la dernière colonne d'un élément du réseau s'écrit donc $10^k(\lambda_1b_1+\lambda_2b_2+...\lambda_nb_n)$. Il existe un tel élément du réseau de dernière colonne nulle ssi $\lambda_1b_1+\lambda_2b_2+...\lambda_nb_n =0$ $\Longleftrightarrow$ les éléments $x_i$ sont entièrement linéairement indépendants.

b)On applique LLL à la base $b_i$ où les $x_i$ sont remplacés par les puissances successives de $x$ et trouver un élément de dernière colonne et conclure comme précédemment. LLL va trouver un élément de dernière colonne nuille puisqu'on lui donne une base de taille dépassant le rang du réseau.

In [54]:
#integer_linear_comb prend comme entrée une suite de nombres réels x_i, ...., x_n et essaye de trouver des entiers lambda_1, ..., lambda_n
# tels que lambda_1 x_1 + .... + lambda_n x_n est très proche à zero (la fonction peut retourner les lambda_1, ..., lambda_n et la valeur
# lambda_1 x_1 + .... + lambda_n x_n minimale trouvée)
#def integer_linear_comb(x, k) :

# algebraic prend comme entrer un nombre réel x et un entier n et vérifie si il existe un polynôme P à coefficients entiers et de degré n
# tel que P(x) est très proche à zero (la fonction peut retourner les cofficients de P et la valeur P(x) minimale trouvé)
#def algebraic(x, n) :


[1 1]
1.4142135623730951


## 2) Algorithme de Babai

Rappelons que le problème du vecteur plus proche ou CVP (Closest Vector Problem) consiste à trouver, pour $\mathcal{L} \subseteq \mathbb{R}^n$ un réseau et $x \in \mathbb{R}^n$, le vecteur de $\mathcal{L}$ qui est plus proche à $x$. Le problème $r$-CVP consiste à trouver un vecteur $v \in \mathcal{L}$ tel que $|| x - v || \leq r d_0$, où $d_0$ dénote la plus courte distance entre $x$ et un vecteur dans $\mathcal{L}$. L'algorithme de Babai permet de calculer une solution au $r-$CVP (Closest Vector Problem) avec un rayon d'approximation $r = 2 \big(\frac{2}{\sqrt{3}}\big)^{n}$.

a) Implementer l'algorithme de Babai

In [40]:
def Babai(b, v) :

IndentationError: expected an indented block after function definition on line 1 (3445431588.py, line 1)

b) En utilisant les bases de 1.c), appliquer l'algorithme aux vecteur suivants ⁉

* $v_1 = (34 , 77); \;\;\;, v_2 = (3.017 , 17.979)$
* $v_1 = (0 , 1500 , 427), v_2 = (0, 999 , 7), v_3 = (10, 10, 10)$.

c) Vérifier si les réponses obtenues par l'algorithme sont optimales.

# 3) Cryptosystème de Merkle-Hellman

Dans cet exercice, on étudie une application de LLL à l'attaque du système de cryptage de Merkle-Hellman.

Rappelons tout d'abord le problème du sac à dos : on a un sac à dos qui peut contenir au plus $C$ (kilogrammes), où $C > 0$ est un entier, et on a $m$ objets de poids $a_1, \ldots, a_n$ (kilogrammes). Le problème consiste à maximiser le poids qu'on peut introduire dans le sac à dos. Autrement dit, trouver des $(m_i)_{1 \leq i \leq n}, m_i \in \{0, 1\}$ tels que $C \leq \sum_{i = 1}^n m_i a_i$ et $\sum_{i = 1}^n m_i a_i$ soit maximal.

a) Supposons que pout tout $i >0$, on a $a_i > \sum_{j = 1}^i a_j$. Démontrer que la solution au problème du sac à dos est unique et donner un algorithme linéaire pour résoudre le problème du sac à dos.

In [ ]:
# La fonction Knapsack prend comme entrée un entier c > 0 et une liste d'entiers a = (a_i), a_i >0 et retourne une suite (c_i)
# avec c_i = 0 ou 1 de sorte que c >= \sum_i c_i a_i et c - \sum_i c_i a_i soit minimal.

def Knapsack(c, a) :

Rappelons brièvement le cryptosystème de Merkle-Hellman basé sur le problème du sac à dos.

* Soit $n \in \mathbb{N}$ tel que le message à codifier peut être représenté comme une suite de $n$ bits. 
* On choisit une suite $(a_i)_{1 \leq i \leq n}$ d'entiers positifs tels que $a_i > \sum_{j < i} a_j$ pour tout $i$ (e.g. $a_i = 2^i$).
* On choisit un entier $N > \sum_{i = 1}^n a_i$ et $K \in (\mathbb{Z} / N \mathbb{Z})^\times$ arbitraires et on pose $b_i = K a_i$ mod $N$. La clé publique est la suite $(b_i)_{1 \leq i \leq n}$. La clé privée est composée de la suite $(a_i)_{1 \leq i \leq n}$ et les entiers $N$ et $K$.
* Le message $m = (m_i)_{1 \leq i \leq n}$, avec $m_i \in \{0, 1\}$ est codifié en $c = \sum_{i = 1}^n m_i b_i$.
* Pour déchiffrer, on résout le problème du sac à dos pour la valeur $K^{-1} c$ mod $N$ et la suite $a_i$ pour obtenir $N = \sum_{i = 1}^n m_i a_i$, où $m = (m_i)_{1 \leq i \leq n}$ est le message original.

b) Faire une implémentation du cryptosystème de Merkle-Hellman.


In [ ]:
# la fonction keys produit la clée privé et la clé secrete du cryptosystème

def knapsack_keys() :

# La fonction knapsack_crypt code le message m en utilisant la clé publique public_key

def knapsack_crypt(m, public_key) :

# La fonction knapsack_decrypt code le message m en utilisant la clé sécrete secret_key

def knapsack_decrypt(m, secret_key) :

c) Cryptoanalyser le système de Merkle-Hellman en utilisant LLL.

In [ ]:
a="moussa;d"
a.split()